In [1]:
import pandas as pd

from sklearn.metrics import accuracy_score, make_scorer, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold

import sys
import time

sys.path.insert(1, "../src")
from knn import *
from pca import *

N_SPLITS = 10

train = pd.read_csv("../data/train.csv")[:1000]

X = train.drop(columns='label').to_numpy()
y = train.label.to_numpy()

In [6]:
grid = {
    "k": [2],
    # "k": [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    "alpha": [10],
    # "alpha": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 153],
}

skf = StratifiedKFold(n_splits=N_SPLITS)
splits = list(skf.split(X, y))

metrics = dict()

for k in grid["k"]:
    for a in grid["alpha"]:
        
        score_values = []
        time_values = []
        print(f"Running gridsearch for: k={k} - alpha={a}")
        
        print("\tSplits:")
        for idx, (train_index, test_index) in enumerate(splits):
            print(f"\t\t{idx} time:", end="\t\t")
            _x_train, _y_train = X[train_index], y[train_index]
            _x_test, _y_test = X[test_index], y[test_index]
            
            _pca, _knn = PCA(alpha=a), KNNClassifier(k=k)
            
            _st = time.time()
            
            _pca.fit(_x_train)
            _x_train_transformed = _pca.transform(_x_train)
            _knn.fit(_x_train_transformed, _y_train)
            _x_test_transformed = _pca.transform(_x_test)
            _pred = _knn.predict(_x_test_transformed)
            
            _et = time.time()
            
            print(f"{(_et - _st):.4f}")
            
            _metric = accuracy_score(_y_test, _pred)
            score_values.append(_metric)
            time_values.append(_et - _st)
        
        print("")
        
        metrics[f"{k} - {a}"] = dict()
        metrics[f"{k} - {a}"]['score'] = np.mean(score_values)
        metrics[f"{k} - {a}"]['time'] = np.mean(time_values)
        print(np.mean(score_values))

Running gridsearch for: k=2 - alpha=10
	Splits:
		0 time:		5.9413
		1 time:		6.9426
		2 time:		6.1366
		3 time:		6.0918
		4 time:		5.2989
		5 time:		

KeyboardInterrupt: 